# Secret Visium-HD Pre-process

In [ ]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import celldega as dega

In [ ]:
import os
import json

In [ ]:
from http.server import HTTPServer, SimpleHTTPRequestHandler
import threading as thr
import requests as rq

In [ ]:
# class CORSHTTPRequestHandler(SimpleHTTPRequestHandler):
#     def end_headers(self):
#         self.send_header('Access-Control-Allow-Origin', '*')
#         self.send_header('Access-Control-Allow-Methods', 'GET, OPTIONS')
#         self.send_header('Access-Control-Allow-Headers', 'X-Requested-With, content-type, Authorization')  # Added Authorization
#         self.send_header('Access-Control-Allow-Credentials', 'true')
#         super().end_headers()

#     def do_OPTIONS(self):
#         self.send_response(200)
#         self.end_headers()

#     def log_message(self, format, *args):
#         # Override log_message to prevent logging to the console.
#         pass


# server = HTTPServer(("", 0), CORSHTTPRequestHandler)
# print(f"Server running on port {server.server_address[1]}")

# service = thr.Thread(target=server.serve_forever)
# service.start()

# server_address = server.server_address[1]
# server_address

In [ ]:
# base_path = 'secret_Visium-HD/Visium_HD_Human_Lung_Cancer/square_016um/'
base_path = 'secret_Visium-HD/Visium_HD_Human_Lung_Cancer/square_008um/'

In [ ]:


# # there is no analysis directory at 2um resolution
# base_path = 'secret_Visium-HD/Visium_HD_Human_Lung_Cancer/square_002um/'

df_pos = pd.read_parquet(base_path + 'spatial/tissue_positions.parquet')
# df_pos.head(50)

In [ ]:
# # Determine average row and column spacing (if not known)
# average_row_spacing = df_pos['pxl_row_in_fullres'].diff().median()
# average_col_spacing = df_pos['pxl_col_in_fullres'].diff().median()

# # Clean up the coordinates by rounding to the nearest grid position
# df_pos['pxl_row_in_fullres_rounded'] = (df_pos['pxl_row_in_fullres'] / average_row_spacing).round() * average_row_spacing
# df_pos['pxl_col_in_fullres_rounded'] = (df_pos['pxl_col_in_fullres'] / average_col_spacing).round() * average_col_spacing

# # # Save the cleaned data if needed
# # df.to_csv('cleaned_positions.csv', index=False)

In [ ]:
# # Calculate the dimensions of the grid
# max_row = df_pos['array_row'].max() + 1
# max_col = df_pos['array_col'].max() + 1

# # Assuming uniform dimensions for each tile
# tile_width = 25.0  # Width of each tile, determine as needed
# tile_height = 25.0  # Height of each tile, determine as needed

# # Calculate the starting point (top left)
# start_x = df_pos['pxl_col_in_fullres'].min()
# start_y = df_pos['pxl_row_in_fullres'].min()

# # Assign new coordinates based on a perfect grid layout
# df_pos['pxl_row_in_fullres_aligned'] = start_y + df['array_row'] * tile_height
# df_pos['pxl_col_in_fullres_aligned'] = start_x + df['array_col'] * tile_width

In [ ]:
df_pos = df_pos[df_pos['in_tissue'] == 1]

In [ ]:
df_pos['in_tissue'].value_counts()

In [ ]:
df_pos.head()

In [ ]:
df_pos.shape[0]/1e6

In [ ]:
# Plotting
plt.figure(figsize=(10, 8))
plt.scatter(df_pos['pxl_col_in_fullres'], df_pos['pxl_row_in_fullres'], c='blue', alpha=0.6, edgecolors='w', s=2)
plt.title('Tile Positions in Spatial Transcriptomics Assay')
plt.xlabel('Pixel Column in Full Resolution')
plt.ylabel('Pixel Row in Full Resolution')
plt.gca().invert_yaxis()  # Invert y-axis to match the image coordinates
plt.grid(True)
plt.show()

In [ ]:
import json

In [ ]:
filepath_scalefactors = base_path + 'spatial/scalefactors_json.json'
with open(filepath_scalefactors) as file:
    data = json.load(file)

print(data)

In [ ]:
scale_factor = data['tissue_hires_scalef']

In [ ]:
df_pos['x_scaled'] = df_pos['pxl_col_in_fullres'] * scale_factor
df_pos['y_scaled'] = df_pos['pxl_row_in_fullres'] * scale_factor

In [ ]:
print(df_pos.x_scaled.min(), df_pos.x_scaled.max())
print(df_pos.y_scaled.min(), df_pos.y_scaled.max())

In [ ]:
df_pos.shape

In [ ]:
# Sample 10% of the rows randomly
df_sample = df_pos.sample(frac=0.01)

In [ ]:
df_sample.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image

In [ ]:
image_path = base_path + 'spatial/tissue_hires_image.png'
img = Image.open(image_path)

In [ ]:
# Create a plot with the image
fig, ax = plt.subplots(figsize=(10, 12))
ax.imshow(img)

# Scatter plot of the sampled data points
ax.scatter(df_sample['x_scaled'], df_sample['y_scaled'], color='red', s=0.1, alpha=1)

# Display the plot
plt.show()


In [ ]:
meta_tile = pd.read_csv(base_path + 'analysis/clustering/gene_expression_graphclust/clusters.csv')
meta_tile

In [ ]:
from matplotlib.colors import to_hex
# Get all categorical color palettes from Matplotlib and flatten them into a single list of colors
palettes = [plt.get_cmap(name).colors for name in plt.colormaps() if "tab" in name]
flat_colors = [color for palette in palettes for color in palette]

# Convert RGB tuples to hex codes
flat_colors_hex = [to_hex(color) for color in flat_colors]

In [ ]:
meta_tile['Cluster'].value_counts()

In [ ]:
clusters = [str(x) for x in meta_tile['Cluster'].unique().tolist()]

# Use modular arithmetic to assign a color to each gene, white for genes with "Blank"
colors = [flat_colors_hex[i % len(flat_colors_hex)] if "Blank" not in cluster else "#FFFFFF" for i, cluster in enumerate(clusters)]

# Create a DataFrame with genes and their assigned colors
df_colors = pd.DataFrame({'color': colors}, index=clusters)

df_colors.to_parquet(base_path + 'landscape_files/df_colors.parquet')

In [ ]:
df_colors.head()

In [ ]:
# meta_tile_json = meta_tile.to_json(orient='records')
meta_tile_json = dict(zip(meta_tile['Barcode'], meta_tile['Cluster']))

json_output = json.dumps(meta_tile_json)

with open(base_path + 'landscape_files/meta_tile.json', 'w') as file:
    file.write(json_output)

In [ ]:
path_output = base_path + 'landscape_files'
if not os.path.exists(path_output):
    os.mkdir(path_output)

In [ ]:
tile = pd.DataFrame()
tile['name'] = df_pos['barcode']

# tile['center_x'] = df_pos['pxl_col_in_fullres'].round(1)
# tile['center_y'] = df_pos['pxl_row_in_fullres'].round(1)

tile['center_x'] = df_pos['x_scaled'] # .round(1)
tile['center_y'] = df_pos['y_scaled'] # .round(1)

tile['center_x'] = tile['center_x'].astype(float)
tile['center_y'] = tile['center_y'].astype(float)

In [ ]:
path_tile_geo = base_path + 'landscape_files/tile_geometries.parquet'

In [ ]:
from copy import deepcopy

In [ ]:
tile_merge = deepcopy(tile)
tile_merge.index = tile['name'].values.tolist()

In [ ]:
tile_merge.head()

In [ ]:
meta_tile_merge = deepcopy(meta_tile)

In [ ]:
meta_tile_merge.index = meta_tile['Barcode'].values.tolist()
meta_tile_merge.head()

In [ ]:
cats = [str(x) for x in meta_tile_merge['Cluster'].values.tolist()]

In [ ]:
keep_tiles = meta_tile_merge.index.tolist()
tile_merge.loc[keep_tiles, 'cluster'] = pd.Series(cats, index=keep_tiles)

In [ ]:
tile_merge

In [ ]:
tile_merge.to_parquet(base_path + 'landscape_files/meta_tile.parquet')

In [ ]:
tile_merge.center_x.mean()

In [ ]:
tile_merge.center_y.mean()

In [ ]:
tile_merge['geometry'] = tile_merge.apply(lambda row: [row['center_x'], row['center_y']], axis=1)
tile_merge.loc[keep_tiles, ['name', 'geometry', 'cluster']].to_parquet(path_tile_geo) 

In [ ]:
tile_merge

In [ ]:
pd.read_parquet(path_tile_geo).head()

In [ ]:
!pip install h5py

In [ ]:
import h5py

In [ ]:
file_path = 'secret_Visium-HD/Visium_HD_Human_Lung_Cancer/square_008um/filtered_feature_bc_matrix.h5'

In [ ]:
def print_structure(file, indent = ' '):
    """Recursively prints the groups and datasets in an HDF5 file."""
    for key in file.keys():
        item = file[key]
        print(indent, key)
        if isinstance(item, h5py.Dataset):
            print(indent + ' ', "Dataset")
        elif isinstance(item, h5py.Group):
            print_structure(item, indent + ' ')
            
with h5py.File(file_path, 'r') as file:
    print_structure(file)


In [ ]:
# import h5py

# # Function to load an HDF5 file
# def load_hdf5_file(file_path):
#     return h5py.File(file_path, 'r')

# # Usage
# file_path = 'secret_Visium-HD/Visium_HD_Human_Lung_Cancer/square_008um/filtered_feature_bc_matrix.h5'
# hdf5_file = load_hdf5_file(file_path)


In [ ]:
# !pip install scipy

In [ ]:
# import h5py
# import pandas as pd
# from scipy.sparse import csr_matrix

# def load_expression_data(file_path):
#     with h5py.File(file_path, 'r') as file:
#         # Access the dataset components of the sparse matrix
#         data = file['matrix/data'][:]
#         indices = file['matrix/indices'][:]
#         indptr = file['matrix/indptr'][:]
#         shape = file['matrix/shape'][:]

#         # Construct the CSR matrix
#         expression_matrix = csr_matrix((data, indices, indptr), shape=tuple(shape))

#         # Optional: Convert to DataFrame for easier handling, assuming gene names are loaded similarly
#         genes = file['matrix/features/name'][:]
#         genes = [gene.decode('utf-8') for gene in genes]  # Decode byte strings
#         gene_expression_df = pd.DataFrame(expression_matrix.toarray(), columns=genes)

#     return gene_expression_df

# # Use the function
# file_path = 'secret_Visium-HD/Visium_HD_Human_Lung_Cancer/square_008um/filtered_feature_bc_matrix.h5'
# gene_expression_df = load_expression_data(file_path)
# print(gene_expression_df.head())  # Preview the first few rows of the DataFrame


In [ ]:
# with h5py.File(file_path, 'r') as file:
#     shape = file['matrix/shape'][:]  # Ensure this is correct
#     print("Matrix shape from file:", shape)


In [ ]:
# def load_expression_data(file_path):
#     with h5py.File(file_path, 'r') as file:
#         data = file['matrix/data'][:]
#         indices = file['matrix/indices'][:]
#         indptr = file['matrix/indptr'][:]
#         shape = file['matrix/shape'][:]
        
#         print("Data loaded, constructing matrix with shape:", shape)
        
#         # Assuming shape is stored as [cols, rows] and needs to be reversed
#         if len(shape) == 2:
#             shape = tuple(reversed(shape))  # Correct shape if necessary
        
#         expression_matrix = csr_matrix((data, indices, indptr), shape=shape)

#         genes = file['matrix/features/name'][:]
#         genes = [gene.decode('utf-8') for gene in genes]

#         barcodes = file['matrix/barcodes'][:]
#         barcodes = [barcode.decode('utf-8') for barcode in barcodes]
        
#         gene_expression_df = pd.DataFrame(expression_matrix.toarray(), columns=genes, index=barcodes)

#     return gene_expression_df

In [ ]:
# # Use the function
# file_path = 'secret_Visium-HD/Visium_HD_Human_Lung_Cancer/square_008um/filtered_feature_bc_matrix.h5'
# gene_expression_df = load_expression_data(file_path)
# print(gene_expression_df.head())  

# Read Tile-by-Gene Sparse Matrix

In [ ]:
import pandas as pd
from scipy.io import mmread
from scipy.sparse import csr_matrix

In [ ]:
def make_unique(names):
    from collections import Counter
    counts = Counter(names)
    new_names = []
    used_names = {}
    
    for name in names:
        if counts[name] > 1:  # If this name is a duplicate
            if name in used_names:
                used_names[name] += 1
            else:
                used_names[name] = 1
            new_name = f"{name}-{used_names[name]}"
        else:
            new_name = name
        new_names.append(new_name)
    
    return new_names

In [ ]:
%%time
# File paths
barcodes_path = 'secret_Visium-HD/Visium_HD_Human_Lung_Cancer/square_008um/filtered_feature_bc_matrix/barcodes.tsv.gz'
features_path = 'secret_Visium-HD/Visium_HD_Human_Lung_Cancer/square_008um/filtered_feature_bc_matrix/features.tsv.gz'
matrix_path = 'secret_Visium-HD/Visium_HD_Human_Lung_Cancer/square_008um/filtered_feature_bc_matrix/matrix.mtx.gz'

# Read barcodes and features
barcodes = pd.read_csv(barcodes_path, header=None, compression='gzip')
features = pd.read_csv(features_path, header=None, compression='gzip', sep='\t')

# Read the gene expression matrix and transpose it
matrix = mmread(matrix_path).transpose().tocsc()  # Transpose and convert to CSC

# Create a sparse DataFrame with genes as columns and barcodes as rows
tbg = pd.DataFrame.sparse.from_spmatrix(matrix, index=barcodes[0], columns=features[1])

tbg.columns = make_unique(tbg.columns.tolist())

print(tbg.shape)

In [ ]:
meta_tile = pd.read_parquet(base_path + 'landscape_files/meta_tile.parquet')
meta_tile.head()

In [ ]:
tbg.head()

In [ ]:
gene_exp = tbg.sum()

In [ ]:
# expressed_tiles = (tbg > 0).astype(int).sum()

In [ ]:
expressed_tiles

In [ ]:
import numpy as np

In [ ]:
%%time
# Calculate mean expression across tiles with float precision
print('calculating mean expression from sparse float data')
mean_expression = tbg.astype(pd.SparseDtype("float", 0)).mean(axis=0)

# Calculate the variance as the average of the squared deviations
print('calculating variance by looping over rows')
num_tiles = tbg.shape[1]
variance = tbg.apply(lambda x: ((x - mean_expression[x.name]) ** 2).sum() / num_tiles, axis=0)
std_deviation = np.sqrt(variance)

# # Assuming mean_expression is aligned with columns
# print('calculating variance using vectorized approach')
# squared_deviations = (tbg - mean_expression) ** 2
# variance = squared_deviations.mean(axis=0)

# Calculate maximum expression
max_expression = tbg.max(axis=0)

# Calculate proportion of tiles with non-zero expression
proportion_nonzero = (tbg != 0).sum(axis=0) / len(tbg)

# Create a DataFrame to hold all these metrics
meta_gene = pd.DataFrame({
    'mean': mean_expression,
    'std': std_deviation,
    'max': max_expression,
    'non-zero': proportion_nonzero
})

In [ ]:
meta_gene.head()

In [ ]:
meta_gene['max'].sort_values(ascending=False).head(10)

In [ ]:
# Convert only the sparse columns to dense
for column in meta_gene.columns:
    # Check if the column dtype is an instance of pd.SparseDtype
    if isinstance(meta_gene[column].dtype, pd.SparseDtype):
        meta_gene[column] = meta_gene[column].sparse.to_dense()

In [ ]:
# # Convert only the sparse columns to dense
# for column in meta_gene.columns:
#     if pd.api.types.is_sparse(meta_gene[column].dtype):
#         meta_gene[column] = meta_gene[column].sparse.to_dense()


In [ ]:
meta_gene.to_parquet(base_path + 'landscape_files/meta_gene.parquet')

In [ ]:
%%time
mean_expression = tbg.astype(pd.SparseDtype("float", 0)).mean(axis=0)
mean_expression

In [ ]:
mean_expression.sort_values(ascending=False).head(5)

In [ ]:
# tbg.shape

In [ ]:
# tbg.head()

In [ ]:
# pd.Series(tbg.columns.tolist()).value_counts()

In [ ]:
# gene_exp = tbg.sum()

In [ ]:
# gene_exp.sort_values(ascending=False)

In [ ]:
# tbg.loc[keep_tiles, 'IGKC'].hist(bins=100)

In [ ]:
base_path

# save parquet files

In [ ]:
# import os

# output_dir = base_path + 'landscape_files/tbg/'
# os.makedirs(output_dir, exist_ok=True)  

# for index, gene in enumerate(cbg.columns):
#     if index % 100 == 0:
#         print(index)
#     # Extract the column as a DataFrame
#     col_df = tbg[[gene]].copy()

#     col_df = col_df.sparse.to_dense()
#     # col_df.fillna(0, inplace=True)    
#     col_df = col_df.astype(int)

#     inst_df = pd.DataFrame(col_df.values, columns=[gene], index=col_df.index.tolist())    

#     inst_df = inst_df.loc[keep_tiles]
#     inst_df.replace(0, pd.NA, inplace=True)
#     inst_df.dropna(how='all', inplace=True)    
    
#     # Save to Parquet
#     inst_df.to_parquet(os.path.join(output_dir, f'{gene}.parquet'))


In [ ]:
# hdf5_file.keys()

In [ ]:
# hdf5_file['matrix'].keys()

In [ ]:
# hdf5_file['matrix']['data']

In [ ]:
# hdf5_file['matrix']['features'].keys()

In [ ]:
# # genes = [x.decode('utf-8') for x in hdf5_file['matrix']['features']['name'][:].tolist()]
# genes[:5]

In [ ]:
# h5py.File(file_path, 'r')

In [ ]:
base_url = f'http://localhost:{server_address}/{base_path}landscape_files/'
base_url

In [ ]:
pd.read_parquet(base_path + 'landscape_files/tbg/SRC.parquet')

In [ ]:
# dega.viz.Toy(base_url=base_url)

In [ ]:
# tbg

In [ ]:
# # base_path = 'secret_Visium-HD/Visium_HD_Human_Lung_Cancer/square_016um/'
# base_path = 'secret_Visium-HD/Visium_HD_Human_Lung_Cancer/square_008um/'

# # # there is no analysis directory at 2um resolution
# # base_path = 'secret_Visium-HD/Visium_HD_Human_Lung_Cancer/square_002um/'

# df_pos = pd.read_parquet(base_path + 'spatial/tissue_positions.parquet')
# # df_pos.head(50)

In [ ]:
# tile_merge

### Make Image Pyramid

In [ ]:
# image_data_scaled = image_data[:,:] * 5

# # Save the image data to a regular TIFF file without compression
# tifffile.imwrite(base_path + 'output_regular.tif', image_data_scaled, compression=None)
# image_ds = dega.pre.reduce_image_size(base_path + 'output_regular.tif', 0.5, path_landscape_files)
# image_jpeg = dega.pre.convert_to_jpeg(image_ds, quality=80)
# dega.pre.make_deepzoom_pyramid(image_jpeg, base_path + 'landscape_files/pyramid_images/', 'cellbound')

In [ ]:
base_path

In [ ]:
dega.pre.make_deepzoom_pyramid(base_path + 'spatial/tissue_hires_image.png', base_path + 'landscape_files/pyramid_images/', 'cells', suffix=".jpeg[Q=95]")

In [ ]:
# Example usage:
path_image_pyramid = base_path + 'landscape_files/pyramid_images/cells_files/'  # Change this to your actual directory path
max_pyramid_zoom = dega.pre.get_max_zoom_level(path_image_pyramid)

print(max_pyramid_zoom)

In [ ]:
dega.pre.save_landscape_parameters(
    'Visium-HD', 
    base_path + '/landscape_files/',
    'cells_files',
    tile_size=100    
)